
# ICD-11 Embedding Models Evaluation with Non Medical Terms

This notebook evaluates how different embedding models perform when mapping
everyday, non-medical descriptions of health issues to standardized ICD-11 codes.
We're using the actual model implementations to get authentic results.

Why this matters:
- Each model was trained differently, affecting how it understands medical language
- BERT models use transformer architectures that capture context
- FastText uses word-level embeddings that can handle misspellings
- TF-IDF relies on term frequency, making it interpretable but less semantic

In [3]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
#! pip install gensim
#from gensim.models import FastText
from typing import List, Dict, Tuple
import json
from tqdm import tqdm
import warnings
import re
warnings.filterwarnings('ignore')

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


These are the actual pre-trained models we'll use. Each has different strengths:
 - BERT: General language understanding
 - BioBERT: Trained on biomedical literature
 - BioClinicalBERT: Trained on clinical notes
 - PubMedBERT: Trained on PubMed abstracts
 - GatorTron: Large-scale clinical language model
 - FastText: Word embeddings that handle morphological variations
 - TF-IDF: Statistical approach based on term importance


In [17]:
# Expected ICD categories for validation
import pandas as pd

EXPECTED_CATEGORIES = [
    "burn",           # T20-T25 Burns
    "headache",       # G44 Headache syndromes
    "respiratory",    # J00-J99 Respiratory diseases
    "sprain",         # S93 Sprains
    "gastro",         # K00-K93 Digestive diseases
    "dermatitis",     # L20-L30 Dermatitis
    "laceration",     # S61 Open wounds
    "anxiety",        # F40-F48 Anxiety disorders
    "dorsalgia",      # M54 Dorsalgia
    "infection"       # A00-B99 Infectious diseases
]

# Repeat each category 30 times and flatten the list
repeated_categories = [category for category in EXPECTED_CATEGORIES for _ in range(30)]

# Convert to pandas Series
category_series = pd.Series(repeated_categories)


In [39]:
# === 1. Imports ===
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from typing import List

# === 2. Utility: Parse stringified vectors ===
def _parse_vec(cell):
    if isinstance(cell, str):
        try:
            return np.array(ast.literal_eval(cell), dtype=float)
        except:
            parts = cell.strip('[]').replace(',', ' ').split()
            return np.array([float(x) for x in parts], dtype=float)
    elif isinstance(cell, (list, np.ndarray)):
        return np.array(cell, dtype=float)
    else:
        raise ValueError(f"Unrecognized vector format: {type(cell)}")

# === 3. Evaluation function with SVD ===
def evaluate_model(
    icd_df: pd.DataFrame,
    prompt_df: pd.DataFrame,
    icd11_lookup_df: pd.DataFrame,
    top_k: int = 3,
    expected_categories: List[str] = None
) -> pd.DataFrame:
    """
    Evaluate prompts against ICD11 embeddings using cosine similarity.
    Automatically applies SVD to reduce dimensionality if needed.
    Returns a long-form DataFrame with results and prints top matches.
    """
    icd_df = icd_df.copy()
    prompt_df = prompt_df.copy()
    
    icd_df.columns = icd_df.columns.str.strip()
    prompt_df.columns = prompt_df.columns.str.strip()

    if "Prompt" not in prompt_df.columns:
        raise KeyError("'Prompt' column not found in prompt_df.")
    if "Vector" not in icd_df.columns or "Vector" not in prompt_df.columns:
        raise KeyError("Missing 'Vector' column in one of the dataframes.")

    icd_df["vector"] = icd_df["Vector"].apply(_parse_vec)
    prompt_df["vector"] = prompt_df["Vector"].apply(_parse_vec)

    icd_vectors = np.vstack(icd_df["vector"].values)
    prompt_vectors = np.vstack(prompt_df["vector"].values)

    # Dimensionality adjustment using SVD
    d_icd = icd_vectors.shape[1]
    d_prompt = prompt_vectors.shape[1]
    
    if d_icd != d_prompt:
        min_dim = min(d_icd, d_prompt)
        print(f"[INFO] Reducing ICD ({d_icd}D) and Prompt ({d_prompt}D) vectors to {min_dim}D using TruncatedSVD...")
        svd_icd = TruncatedSVD(n_components=min_dim, random_state=42)
        svd_prompt = TruncatedSVD(n_components=min_dim, random_state=42)
        icd_vectors = svd_icd.fit_transform(icd_vectors)
        prompt_vectors = svd_prompt.fit_transform(prompt_vectors)

    icd_codes = icd_df["ICD11_code"].tolist()
    prompt_texts = prompt_df["Prompt"].tolist()

    out_rows = []
    for i, (prompt, pvec) in enumerate(zip(prompt_texts, prompt_vectors)):
        sims = cosine_similarity([pvec], icd_vectors)[0]
        top_indices = sims.argsort()[::-1][:top_k]

        print(f"\nPrompt {i+1}: '{prompt}'")
        if expected_categories:
            print(f"Expected category: {expected_categories[i]}")
        print("\nTop 3 matches:")

        for rank, idx in enumerate(top_indices, start=1):
            icd_code = icd_codes[idx]
            row = icd11_lookup_df[icd11_lookup_df["code"] == icd_code].iloc[0]
            title = row.get("title", "(title not found)")
            definition = row.get("definition", "")
            similarity = float(sims[idx])

            print(f"{rank}. {icd_code} – {title}")
            print(f"   Similarity: {similarity:.3f}")
            if definition:
                print(f"   Definition: {str(definition)[:100]}...")

            out_rows.append({
                "prompt_idx": i,
                "prompt_text": prompt,
                "rank": rank,
                "ICD11_code": icd_code,
                "title": title,
                "similarity": similarity,
                "definition": definition
            })

    return pd.DataFrame(out_rows)

# === 5. Model setup ===
#models = ["bert", "biobert", "bioclinicalbert", "pubmedbert"]

models = ["tfidf", "fasttext", "bert", "biobert", "bioclinicalbert", "pubmedbert", "gatortron"]

# === 6. Load ICD11 lookup table ===
lookup_df = pd.read_csv("ICD11_preprocessed.csv")

# === 7. Load embeddings for all models ===
embeddings_ICD11 = {}
embeddings_prompts = {}

for model in models:
    icd_path = f"{model}_ICD11_embeddings.csv"
    prompt_path = f"{model}_non_medical_prompts_embeddings.csv"
    
    icd_df = pd.read_csv(icd_path)
    prompt_df = pd.read_csv(prompt_path)
    prompt_df["Prompt"] = category_series

    embeddings_ICD11[model] = icd_df
    embeddings_prompts[model] = prompt_df

# === 8. Run evaluation for all models ===
all_results = {}

for model in models:
    print(f"\n--- Running evaluation for {model} ---")
    try:
        df = evaluate_model(
            icd_df=embeddings_ICD11[model],
            prompt_df=embeddings_prompts[model],
            icd11_lookup_df=lookup_df,
            top_k=10
        )
        all_results[model] = df
        df.to_csv(f"{model}_prompt_eval.csv", index=False)
    except Exception as e:
        print(f"❌ Evaluation failed for {model}: {e}")



--- Running evaluation for tfidf ---
[INFO] Reducing ICD (768D) and Prompt (136D) vectors to 136D using TruncatedSVD...

Prompt 1: 'burn'

Top 3 matches:
1. LD24.21 – Exostoses with anetodermia and brachydactyly type E
   Similarity: 0.468
2. FA31.0 – Valgus deformity, not elsewhere classified
   Similarity: 0.465
3. 2B50.1 – Chondrosarcoma of bone or articular cartilage of pelvis
   Similarity: 0.462
4. FB31.0 – Progressive osseous heteroplasia
   Similarity: 0.458
5. FA31.2 – Flexion deformity
   Similarity: 0.456
6. NC93.22 – Medial dislocation of proximal end of tibia
   Similarity: 0.452
7. FB82.1 – Osteochondrosis or osteochondritis dissecans
   Similarity: 0.447
8. FB82 – Chondropathies
   Similarity: 0.446
9. LD26.1 – Complex brachydactylies
   Similarity: 0.446
10. LB90 – Joint formation defects
   Similarity: 0.445

Prompt 2: 'burn'

Top 3 matches:
1. KA82.6 – Cerebellar nontraumatic, hemispheres or vermis or posterior fossa haemorrhage of fetus or newborn
   Similarity: 0.5


Prompt 15: 'burn'

Top 3 matches:
1. EK92 – Histiocytoses of uncertain malignant potential
   Similarity: 0.422
2. 2A81.4 – T-cell/histiocyte rich large B-cell lymphoma
   Similarity: 0.412
3. 2A83.3 – Extraosseous plasmacytoma
   Similarity: 0.405
4. 8B60.5 – Amyotrophic lateral sclerosis-Plus
   Similarity: 0.401
5. 2A81.1 – Intravascular large B-cell lymphoma
   Similarity: 0.391
6. BD92 – Lymphangiectasia
   Similarity: 0.387
7. SG70 – Upper energizer stage patterns (TM1)
   Similarity: 0.387
8. SK55 – Weakness of thighs due to vata disorder (TM2)
   Similarity: 0.385
9. 2B31.7 – Fibroblastic reticular cell tumour
   Similarity: 0.382
10. 2B31.6 – Indeterminate cell histiocytosis
   Similarity: 0.381

Prompt 16: 'burn'

Top 3 matches:
1. PA31 – Unintentional railway transport injury event without collision or derailment
   Similarity: 0.530
2. PF5E – Land transport nontraffic injury event of undetermined intent injuring a rider of an animal
   Similarity: 0.500
3. PF4A – Land tran


Prompt 29: 'burn'

Top 3 matches:
1. MC86 – Precordial pain
   Similarity: 0.360
2. BB42 – Periendocarditis
   Similarity: 0.357
3. MD30.1 – Other chest pain
   Similarity: 0.333
4. MD30 – Pain in throat or chest
   Similarity: 0.329
5. NC96.01 – Laceration of Achilles tendon
   Similarity: 0.328
6. MD30.0 – Chest pain on breathing
   Similarity: 0.327
7. MD31 – Pleurisy
   Similarity: 0.325
8. ME86.4 – Symptom or complaint of the elbow
   Similarity: 0.317
9. BA40.1 – Stable angina
   Similarity: 0.317
10. NC36.7 – Injury of bursa of elbow
   Similarity: 0.316

Prompt 30: 'burn'

Top 3 matches:
1. EH40.01 – Disseminated infantile seborrhoeic dermatitis
   Similarity: 0.557
2. EA93 – Pityriasis lichenoides
   Similarity: 0.550
3. SN4A – Psoriasis disorder (TM2)
   Similarity: 0.540
4. SN91 – Dandruff disorder (TM2)
   Similarity: 0.537
5. EA90.41 – Acropustulosis of Hallopeau
   Similarity: 0.537
6. ED90.02 – Phymatous rosacea
   Similarity: 0.535
7. EA94 – Pityriasis rubra pilaris
  

6. 9B50 – Episcleritis
   Similarity: 0.407
7. 6B66 – Depersonalization-derealization disorder
   Similarity: 0.401
8. 9D43 – Impairment of contrast vision
   Similarity: 0.399
9. QA02.3 – Observation for suspected mental or behavioural disorders, ruled out
   Similarity: 0.397
10. 9A7B – Sclerosing keratitis
   Similarity: 0.394

Prompt 44: 'headache'

Top 3 matches:
1. NA07.33 – Diffuse injury of multiple parts of brain
   Similarity: 0.431
2. PA31 – Unintentional railway transport injury event without collision or derailment
   Similarity: 0.426
3. MA16 – Cytological findings in blood, blood-forming organs, or the immune system
   Similarity: 0.419
4. NA05.0 – Injury of intracranial vessels of head
   Similarity: 0.419
5. NA07.32 – Diffuse injury of brainstem
   Similarity: 0.418
6. NA07.30 – Diffuse injury of cerebrum
   Similarity: 0.415
7. NA08.2 – Crushing injury of skull
   Similarity: 0.410
8. 2A20.01 – Chronic myelogenous leukaemia, Philadelphia chromosome (Ph1) positive
   S


Prompt 56: 'headache'

Top 3 matches:
1. BC43.11 – Non-obstructive hypertrophic cardiomyopathy
   Similarity: 0.445
2. BC43.12 – Obstructive hypertrophic cardiomyopathy
   Similarity: 0.442
3. KB40.0 – Neonatal cardiac failure due to pulmonary overperfusion
   Similarity: 0.433
4. SK65 – Cataract disorder (TM2)
   Similarity: 0.428
5. BC43.3 – Endocardial fibroelastosis
   Similarity: 0.417
6. LA8A.6 – Congenital subpulmonary stenosis
   Similarity: 0.415
7. BD10 – Congestive heart failure
   Similarity: 0.414
8. 8C73.0 – Autosomal recessive cardiomyopathy or ophthalmoplegia
   Similarity: 0.413
9. BA01 – Hypertensive heart disease
   Similarity: 0.413
10. 9C84.1 – Congenital forms of nystagmus
   Similarity: 0.413

Prompt 57: 'headache'

Top 3 matches:
1. SF61 – Heart blood deficiency pattern (TM1)
   Similarity: 0.464
2. SF04 – Blood dryness pattern (TM1)
   Similarity: 0.455
3. KB42 – Persistent pulmonary hypertension of the newborn
   Similarity: 0.429
4. SF01 – Blood stasis patte

...


Prompt 222: 'anxiety'

Top 3 matches:
1. ME63.6 – Chronic discoid or annular skin disorder
   Similarity: 0.958
2. EE80.0 – Granuloma annulare
   Similarity: 0.956
3. ME63.2 – Chronic papular skin disorder
   Similarity: 0.954
4. EB11 – Annular erythema
   Similarity: 0.953
5. EA91.1 – Hypertrophic lichen planus
   Similarity: 0.952
6. 2B31.0 – Juvenile xanthogranuloma
   Similarity: 0.951
7. 2B03.1 – Lymphomatoid papulosis
   Similarity: 0.950
8. EB44 – Dermatitis herpetiformis
   Similarity: 0.950
9. DA42.2 – Eosinophilic gastritis
   Similarity: 0.950
10. EA91.2 – Follicular lichen planus
   Similarity: 0.949

Prompt 223: 'anxiety'

Top 3 matches:
1. ND56.9 – Injury complicating pregnancy
   Similarity: 0.930
2. KA01.6 – Fetus or newborn affected by maternal death
   Similarity: 0.930
3. PL13.3 – Incorrect substance, as mode of injury or harm
   Similarity: 0.929
4. KA06.4 – Fetus or newborn affected by maternal use of nutritional chemical substances
   Similarity: 0.927
5. PC99 – 


Prompt 239: 'anxiety'

Top 3 matches:
1. LA90.13 – Cystic hygroma in fetus
   Similarity: 0.940
2. KC22.0 – Subcutaneous fat necrosis of the newborn
   Similarity: 0.940
3. KC20.1 – Umbilical cutis or polyp of newborn
   Similarity: 0.938
4. KC40.1 – Neonatal milia
   Similarity: 0.935
5. LB01 – Omphalocele
   Similarity: 0.935
6. KB2K – Pulmonary cysts in newborn
   Similarity: 0.935
7. DB36.10 – Perirectal abscess
   Similarity: 0.934
8. KC21.1 – Neonatal toxic erythema
   Similarity: 0.934
9. 1A91 – Granuloma inguinale
   Similarity: 0.932
10. LB03.0 – Allantoic duct remnants or cysts
   Similarity: 0.932

Prompt 240: 'anxiety'

Top 3 matches:
1. PK81.B – Manipulative therapies associated with injury or harm in therapeutic use
   Similarity: 0.935
2. EL80 – Adverse cutaneous effects of diagnostic procedures
   Similarity: 0.935
3. PK9A.23 – Mobility aids associated with adverse incidents
   Similarity: 0.935
4. MB40.3 – Anaesthesia of skin
   Similarity: 0.935
5. PK97.4 – Ophthalmi


Prompt 254: 'dorsalgia'

Top 3 matches:
1. LD2F.03 – Fetal Valproate Spectrum Disorder
   Similarity: 0.951
2. EB90.10 – Pretibial myxoedema
   Similarity: 0.950
3. 8D44 – Alcohol-related neurological disorders
   Similarity: 0.950
4. KB61 – Transitory neonatal disorders of calcium or magnesium metabolism
   Similarity: 0.950
5. KA06.2 – Fetus or newborn affected by maternal use of alcohol
   Similarity: 0.949
6. DB99.5 – Hepatic encephalopathy
   Similarity: 0.949
7. KB60 – Transitory disorders of carbohydrate metabolism specific to fetus or newborn
   Similarity: 0.949
8. DB96.10 – Primary biliary cholangitis with overlap syndrome
   Similarity: 0.948
9. 5B5A.00 – Dry beriberi
   Similarity: 0.948
10. 8D40 – Neurological disorders due to nutrient deficiency
   Similarity: 0.947

Prompt 255: 'dorsalgia'

Top 3 matches:
1. NA07.83 – Traumatic haemorrhage in brainstem without specification whether primary or secondary
   Similarity: 0.936
2. KA40.04 – Subarachnoid haemorrhage due to bi

8. PK80.D – Ophthalmic procedure associated with injury or harm in therapeutic use
   Similarity: 0.934
9. ED53 – Skin peeling
   Similarity: 0.934
10. PH24 – Threat to breathing by inhalation or ingestion of liquids with undetermined intent
   Similarity: 0.933

Prompt 271: 'infection'

Top 3 matches:
1. 4A00.1 – Defects in the complement system
   Similarity: 0.964
2. EC10 – Genetic syndromes with poikiloderma
   Similarity: 0.959
3. SP5B – Consumptive disorder (TM2)
   Similarity: 0.957
4. MA16.00 – Haemolysis, not elsewhere classified
   Similarity: 0.957
5. LD20 – Syndromes with central nervous system anomalies as a major feature
   Similarity: 0.956
6. 4A00.0 – Functional neutrophil defects
   Similarity: 0.955
7. LD2F – Syndromes with multiple structural anomalies, without predominant body system involvement
   Similarity: 0.955
8. 4A01.3 – Other well-defined immunodeficiency syndromes due to defects in adaptive immunity
   Similarity: 0.955
9. 4B01.0 – Constitutional disorders 


Prompt 287: 'infection'

Top 3 matches:
1. 2F23.0 – Dermatofibroma
   Similarity: 0.933
2. NC70.3 – Contusion of thigh
   Similarity: 0.930
3. EK70.1 – Trichilemmal cyst
   Similarity: 0.930
4. NC30.1 – Contusion of elbow
   Similarity: 0.929
5. EH92.01 – Soft corn
   Similarity: 0.929
6. NC51.10 – Contusion of other parts of wrist or hand
   Similarity: 0.929
7. ED56 – Keratosis pilaris
   Similarity: 0.929
8. SK50 – Scalp affliction disorder (TM2)
   Similarity: 0.928
9. NC96.5 – Injury of bursa of knee
   Similarity: 0.928
10. FB82.00 – Chondromalacia patellae
   Similarity: 0.928

Prompt 288: 'infection'

Top 3 matches:
1. 1B93.1 – Cellulocutaneous plague
   Similarity: 0.885
2. 1B94.0 – Ulceroglandular tularaemia
   Similarity: 0.882
3. 1C1A.0 – Cutaneous listeriosis
   Similarity: 0.878
4. 1B99 – Pasteurellosis
   Similarity: 0.877
5. 1B94 – Tularaemia
   Similarity: 0.874
6. 1C30.2 – Typhus fever due to Rickettsia typhi
   Similarity: 0.873
7. 1C1C.20 – Acute meningococcaemia
 

In [40]:
all_results

{'tfidf':       prompt_idx prompt_text  rank ICD11_code  \
 0              0        burn     1    LD24.21   
 1              0        burn     2     FA31.0   
 2              0        burn     3     2B50.1   
 3              0        burn     4     FB31.0   
 4              0        burn     5     FA31.2   
 ...          ...         ...   ...        ...   
 2995         299   infection     6    ED90.02   
 2996         299   infection     7       EA94   
 2997         299   infection     8       EB30   
 2998         299   infection     9       SN4D   
 2999         299   infection    10       EB42   
 
                                                   title  similarity definition  
 0     Exostoses with anetodermia and brachydactyly t...    0.468006             
 1            Valgus deformity, not elsewhere classified    0.465261             
 2     Chondrosarcoma of bone or articular cartilage ...    0.461686             
 3                      Progressive osseous heteroplasia    0

# Model evaluation

# 7. Comparative Analysis

In [37]:
def analyze_results_from_prompt_column(results: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Evaluate model performance using 'prompt_text' as the expected keyword.
    - Count correct match if the keyword appears in the title of any top-3 predictions.
    - Use top-1 similarity for similarity statistics.

    Parameters:
        results: model_name → long-form DataFrame from evaluate_model()

    Returns:
        pd.DataFrame with accuracy and similarity statistics for each model
    """
    summary_data = []

    for model_name, df in results.items():
        if df is None or df.empty:
            continue

        correct_matches = 0
        similarity_scores = []

        prompt_indices = df["prompt_idx"].unique()
        for idx in prompt_indices:
            preds = df[df["prompt_idx"] == idx].sort_values("rank").head(3)
            expected_keyword = preds["prompt_text"].iloc[0].lower()

            found = any(expected_keyword in str(row.get("title", "")).lower() for _, row in preds.iterrows())
            if found:
                correct_matches += 1

            # Top-1 similarity
            if not preds.empty:
                similarity_scores.append(preds.iloc[0]["similarity"])

        accuracy = correct_matches / len(prompt_indices) * 100
        summary_data.append({
            "Model": model_name,
            "Correct Matches (Top-3)": correct_matches,
            "Accuracy": f"{accuracy:.1f}%",
            "Avg Similarity (Top-1)": f"{np.mean(similarity_scores):.3f}" if similarity_scores else "N/A",
            "Min Similarity": f"{np.min(similarity_scores):.3f}" if similarity_scores else "N/A",
            "Max Similarity": f"{np.max(similarity_scores):.3f}" if similarity_scores else "N/A"
        })

    return pd.DataFrame(summary_data).sort_values("Correct Matches (Top-3)", ascending=False)


In [38]:
summary_df = analyze_results_from_prompt_column(all_results)

print("\n" + "="*60)
print("PERFORMANCE SUMMARY USING PROMPT TEXT AS KEYWORD")
print("="*60)
print(summary_df.to_string(index=False))



PERFORMANCE SUMMARY USING PROMPT TEXT AS KEYWORD
          Model  Correct Matches (Top-3) Accuracy Avg Similarity (Top-1) Min Similarity Max Similarity
bioclinicalbert                       21     7.0%                  0.885          0.857          0.926
           bert                       19     6.3%                  0.665          0.580          0.752
        biobert                       14     4.7%                  0.891          0.866          0.914
     pubmedbert                        6     2.0%                  0.970          0.958          0.980
          tfidf                        4     1.3%                  0.467          0.360          0.638
       fasttext                        0     0.0%                  0.438          0.437          0.438
      gatortron                        0     0.0%                  0.939          0.885          0.964
